In [ ]:
!pip install adversarial-robustness-toolbox

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 18.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.5/30.5 MB 46.0 MB/s eta 0:00:00
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.2.2
    Uninstalling scikit-learn-1.2.2:
      Successfully uninstalled scikit-learn-1.2.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
bigframes 0.20.1 requires scikit-learn>=1.2.2, but you have scikit-learn 1.1.3 which is incompatible.


**from art.estimators.classification import KerasClassifier:** Importa la classe KerasClassifier dalla libreria ART. Questa classe fornisce un wrapper per l'utilizzo di modelli di classificazione di TensorFlow con la libreria ART.

**from art.attacks.evasion import FastGradientMethod**: Importa la classe FastGradientMethod dalla libreria ART. Questa classe implementa l'attacco avversariale chiamato Fast Gradient Method (FGSM), che è un metodo rapido ed efficace per generare immagini avversariali.

**from art.utils import load_cifar10**: Importa la funzione load_cifar10 dalla libreria ART. Questa funzione semplifica il caricamento del dataset CIFAR-10, che verrà utilizzato per addestrare il modello e generare le immagini avversariali.

In [ ]:
import os
import cv2
import numpy as np
import tensorflow as tf
from tensorflow.keras.applications.resnet import preprocess_input
from art.estimators.classification import KerasClassifier
from art.attacks.evasion import FastGradientMethod
from art.utils import load_cifar10

In TensorFlow 2.0 e versioni successive, l'esecuzione accelerata è abilitata per default. Disabilitare l'esecuzione accelerata è necessario in questo contesto poiché ART è progettata per lavorare con sessioni di TensorFlow non accelerate.

In [ ]:
# Disabilita l'esecuzione accelerata di TensorFlow
tf.compat.v1.disable_eager_execution()

Questa parte del codice carica un modello **MobileNetV2 preaddestrato**, lo adatta a **CIFAR-10** sostituendo l'ultimo livello e crea un **classificatore ART** basato su questo modello.

In [ ]:
# Carica il modello preaddestrato su MobileNetV2 senza il livello fully connected
base_model = tf.keras.applications.MobileNetV2(weights='imagenet', input_shape=(32, 32, 3), include_top=False)
base_model.trainable = False

# Aggiungi un nuovo livello per adattarsi a CIFAR-10
model = tf.keras.models.Sequential([
    base_model,
    tf.keras.layers.GlobalAveragePooling2D(),
    tf.keras.layers.Dense(10, activation='softmax')  # 10 classi per CIFAR-10
])

classifier = KerasClassifier(model=model, clip_values=(0, 1), use_logits=False)


Instructions for updating:
Colocations handled automatically by placer.


9406464/9406464 [==============================] - 0s 0us/step


Carichiamo il dataset **CIFAR-10** utilizzando la funzione load_cifar10 di ART e selezionando solo le prime 10 immagini e le corrispondenti etichette di addestramento, per provare in maniera veloce e su poche immagini l'attacco.

In [ ]:
# Carica il dataset CIFAR-10
(x_train, y_train), (_, _), _, _ = load_cifar10()

# Seleziona 14000 immagini
x_sample = x_train[:14000]
y_sample = y_train[:14000]

**attack = FastGradientMethod(estimator=classifier, eps=0.1)**: Crea un oggetto di attacco FGSM specificando il classificatore classifier (il nostro modello) come estimatore e impostando eps a 0.1. L'epsilon (eps) rappresenta la quantità massima di perturbazione che può essere aggiunta a ciascun pixel nell'immagine per generare l'immagine avversaria. Si potrebbero fare varie prove.

**x_adv = attack.generate(x_sample)**: Applica l'attacco alle immagini di esempio (x_sample). L'attacco FGSM modifica le immagini originali aggiungendo una perturbazione basata sul gradiente della funzione di perdita rispetto alle immagini di input. L'obiettivo è creare immagini che ingannino il modello quando classificate, ma che appaiano simili a occhio umano. Il risultato è memorizzato nella variabile x_adv, che conterrà le immagini avversariali.

In [ ]:
# Definisci l'attacco FGSM
attack = FastGradientMethod(estimator=classifier, eps=0.01)

# Applica l'attacco alle immagini di esempio
x_adv = attack.generate(x_sample)

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training_v1.py:2359: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates=self.state_updates,


In [ ]:
# Calcola la percentuale di successo dell'attacco

# Esegui previsioni sulle immagini originali
predictions_original = classifier.predict(x_sample)

# Esegui previsioni sulle immagini avversariali
predictions_adversarial = classifier.predict(x_adv)

success_rate = (predictions_original.argmax(axis=1) != predictions_adversarial.argmax(axis=1)).mean()
print(f"Percentuale di successo dell'attacco: {success_rate * 100}%")


Percentuale di successo dell'attacco: 85.86428571428571%


In [ ]:
# Creazione di array di etichette per immagini attaccate e originali
labels_attacked = np.ones(x_adv.shape[0])  # 1 indica immagini attaccate
labels_original = np.zeros(x_sample.shape[0])  # 0 indica immagini originali

# Salva le immagini attaccate e le etichette corrispondenti
np.savez('attacked_dataset_big.npz', images=x_adv, labels=labels_attacked)

# Salva le immagini originali e le etichette corrispondenti
np.savez('original_dataset_big.npz', images=x_sample, labels=labels_original)


Per caricare il database su altri progetti, basta importare questi file una volta scaricati, attraverso numpy.

```
import numpy as np

# Carica il dataset attaccato
attacked_data = np.load('attacked_dataset.npz')
attacked_images = attacked_data['images']
attacked_labels = attacked_data['labels']

# Carica il dataset originale
original_data = np.load('original_dataset.npz')
original_images = original_data['images']
original_labels = original_data['labels']

```

